In [0]:
import tensorflow as tf
import numpy as np

# 훈련자료 입력 (독립변수 3개)
x_train = np.array([[5.1,3.5,0.2],[4.9,3.0,1.4],[4.7,3.2,0.2],[4.6,3.1,0.2],[5.0,3.6,0.2],
                    [6.7,3.0,2.3],[6.3,2.5,1.9],[6.5,3.0,2.0],[6.2,3.4,2.3],[5.9,3.0,1.8]], dtype=np.float32)
y_train = np.array([[1],[1],[1],[1],[1],[0],[0],[0],[0],[0]], dtype=np.float32)

# tf.data.Dataset 파이프라인을 이용하여 값을 입력
# from_tensor_slices 클래스 매서드를 사용하면 리스트, 넘파이, 텐서플로 자료형에서 데이터셋을 만들 수 있음
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))
W = tf.Variable(tf.zeros([x_train.shape[1], 1]), name='weight')
b = tf.Variable(tf.zeros([1]), name='bias')

# 원소의 자료구조 반환
print(dataset.element_spec)

(TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))


In [0]:
# 로지스틱 회귀분석
def logistic_regression(features):
    hypothesis = tf.sigmoid(tf.matmul(features, W) + b)
    return hypothesis

# 비용함수 계산
def loss_ftn(features, labels):
    hypothesis = logistic_regression(features)
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

# 경사값 계산
def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_ftn(features, labels)
    return tape.gradient(loss_value, [W, b])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [0]:
# 반복 시행 부분 (위의 세 함수가 필요)
EPOCHS = 1200
for step in range(EPOCHS + 1):
    for features, labels in iter(dataset):
        hypothesis = logistic_regression(features)
        grads = grad(hypothesis, features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W, b]))
        if step % 300 == 0:
            print("Iter: {}, Loss: {:.4f}, W1: {:.4f}, W2: {:.4f}, W3: {:.4f}, B:{:.4f}".format(step, loss_ftn(features, labels), W.numpy()[0,0], W.numpy()[1,0], W.numpy()[2,0], b.numpy()[0]))
            print(np.transpose(hypothesis))

# 정확도 계산 함수; 가설값 0.5가 넘으면 모델은 Yes로 판정
def accuracy_ftn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy

# 검증자료 입력
x_test = [[4.6,3.2,0.2],[5.6,2.8,2.0]]
y_test = [[1],[0]]

# 정확도 산출
test_acc = accuracy_ftn(logistic_regression(x_test), y_test)
print('Accuracy: {}%'.format(test_acc * 100))

Iter: 0, Loss: 0.6902, W1: -0.0037, W2: 0.0007, W3: -0.0041, B:0.0000
[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]]
Iter: 300, Loss: 0.3799, W1: -0.1780, W2: 0.5347, W3: -0.7549, B:0.1202
[[0.717007   0.44889867 0.69862115 0.6910809  0.7312075  0.23100275
  0.25030005 0.28065637 0.28894442 0.33536863]]
Iter: 600, Loss: 0.2712, W1: -0.2765, W2: 0.8741, W3: -1.1905, B:0.2030
[[0.83374065 0.45111194 0.8116703  0.8023747  0.84906745 0.14632672
  0.16602424 0.20559786 0.21817039 0.27926847]]
Iter: 900, Loss: 0.2195, W1: -0.3496, W2: 1.1180, W3: -1.4821, B:0.2663
[[0.8907554  0.45813727 0.87023646 0.8613159  0.9042307  0.10622582
  0.12387425 0.16580987 0.18119094 0.24792005]]
Iter: 1200, Loss: 0.1895, W1: -0.4091, W2: 1.3104, W3: -1.6969, B:0.3189
[[0.9226238  0.4675128  0.9045774  0.8965122  0.93403673 0.08372217
  0.09924308 0.14158942 0.15916888 0.22836773]]
Accuracy: 100%
